In [1]:
import pandas as pd

from src.metrics import map_score, mrr_score, ndcg_score, rmse_score
from src.models.mean_rating_baseline import MeanRatingRecommender
from src.utils import train_test_split

In [2]:
ratings = pd.read_table("../data/ratings.dat", sep="::", names=['UserID', 'MovieID', 'Rating', 'Timestamp'], engine='python')
ratings['Timestamp'] = pd.to_datetime(ratings['Timestamp'], unit='s')

train_ratings, test_ratings = train_test_split(ratings, 'Timestamp')

In [3]:
baseline = MeanRatingRecommender()
baseline.train(train_ratings)

users_test = test_ratings.get_rating_matrix().columns
y_pred = baseline.predict(users_test)

In [4]:
map_score_value = map_score(test_ratings, y_pred, top=50)
mrr_score_value = mrr_score(test_ratings, y_pred, top=50)
ndcg_score_value = ndcg_score(test_ratings, y_pred, top=50)
rmse_score_value = rmse_score(test_ratings, y_pred)

print(f'Baseline MAP: {map_score_value}')
print(f'Baseline MRR: {mrr_score_value}')
print(f'Baseline NDCG: {ndcg_score_value}')
print(f'Baseline RMSE: {rmse_score_value}')

Baseline MAP: 0.01045348193833639
Baseline MRR: 0.14809692261122365
Baseline NDCG: 0.023519293463226947
Baseline RMSE: 486.64811481202804
